# Lunch Time Python

## 25.11.2022: spaCy
<img style="width: 600px;" src="https://upload.wikimedia.org/wikipedia/commons/8/88/SpaCy_logo.svg">

[spaCy](https://spacy.io/) is an open-source natural language processing library written in Python and Cython.

spaCy focuses on production usage and is very fast and efficient. It also supports deep learning workflows through interfacing with [TensorFlow](https://www.tensorflow.org/) or [PyTorch](https://pytorch.org/), as well as the transformer model library [Hugging Face](https://github.com/huggingface).

*Press `Spacebar` to go to the next slide (or `?` to see all navigation shortcuts)*

[Lunch Time Python](https://ssciwr.github.io/lunch-time-python/), [Scientific Software Center](https://ssc.iwr.uni-heidelberg.de), [Heidelberg University](https://www.uni-heidelberg.de/)

# 0 What to do with spaCy

spaCy is very powerful for text annotation:
- sentencize and tokenize
- POS (part-of-speech) and lemma
- NER (named entity recognition)
- dependency parsing
- text classification
- morphological analysis
spaCy can also learn new tasks through integraton with your machine learning stack. It also provides multi-task learning with pretrained transformers like [BERT](https://arxiv.org/abs/1810.04805). 
(BERT is used in the google search engine.)


In [1]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_md")
doc = nlp("The Scientific Software Center offers lunch-time Python - an informal way to learn about new Python libraries.")
displacy.render(doc, style="dep")

![Example: NER](https://uploads-ssl.webflow.com/5fdc17d51dc102ed1cf87c05/603e741d3ec1596898ac503e_namedentityextraction.png)

Source: [width ai](https://www.width.ai/post/spacy-nlp-business-tools)

In [ ]:
# first imports
import torch
from torch import nn  # model
from torch import optim  # optimizer
from torchvision import datasets, transforms  # data and data transforms
from torch.utils.data import random_split, DataLoader  # utilities

import numpy as np
import matplotlib.pyplot as plt

# 1 Tensors

In [ ]:
# directly from data
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)

In [ ]:
# from numpy array
np_array = np.array(data)
x_np = torch.from_numpy(np_array)

In [ ]:
# from another tensor
x_ones = torch.ones_like(x_data)  # retains the properties of x_data
print(f"Ones Tensor: \n {x_ones} \n")

x_rand = torch.rand_like(x_data, dtype=torch.float)  # overrides the datatype of x_data
print(f"Random Tensor: \n {x_rand} \n")

In [ ]:
# use tuples to determine tensor dimensions
shape = (
    2,
    3,
)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

print(f"Random Tensor: \n {rand_tensor} \n")
print(f"Ones Tensor: \n {ones_tensor} \n")
print(f"Zeros Tensor: \n {zeros_tensor}")

In [ ]:
# tensor attributes
tensor = torch.rand(3, 4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

In [ ]:
# by default, tensors are created on CPU
# We move our tensor to the GPU if available
if torch.cuda.is_available():
    tensor = tensor.to("cuda")

In [ ]:
# indexing like numpy
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:, 1] = 0
print(tensor)

In [ ]:
# joining tensors
t1 = torch.cat([tensor, tensor, tensor], dim=1)
print(t1)

In [ ]:
# This computes the matrix multiplication between two tensors. y1, y2, y3 will have the same value
y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)

y3 = torch.rand_like(y1)
torch.matmul(tensor, tensor.T, out=y3)


# This computes the element-wise product. z1, z2, z3 will have the same value
z1 = tensor * tensor
z2 = tensor.mul(tensor)

z3 = torch.rand_like(tensor)
torch.mul(tensor, tensor, out=z3)

In [ ]:
# GPU via CUDA
# torch.randn(5).cuda()
# better (more flexible):
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
torch.randn(5).to(device)

# 2 Datasets and DataLoaders

- datasets: stores the samples and their corresponding labels
- DataLoader: wraps an iterable around the Dataset to enable easy access to the samples

In [ ]:
# import and split data
train_data = datasets.MNIST(
    "data", train=True, download=True, transform=transforms.ToTensor()
)
train, val = random_split(train_data, [55000, 5000])
train_loader = DataLoader(train, batch_size=32)
val_loader = DataLoader(val, batch_size=32)

In [ ]:
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_data), size=(1,)).item()
    img, label = train_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

In [ ]:
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

# 3 Coding a neural network

In [ ]:
# in theory easy via stateless approach
# import torch.nn.functional as F

# loss_func = F.cross_entropy

# def model(xb):
#    return xb @ weights + bias

# print(loss_func(model(xb), yb), accuracy(model(xb), yb))
# gets messy quickly!

In [ ]:
# define model via explicit nn.Module class
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28 * 28, 64)
        self.l2 = nn.Linear(64, 64)
        self.l3 = nn.Linear(64, 10)
        self.do = nn.Dropout(0.1)

    def forward(self, x):
        h1 = nn.functional.relu(self.l1(x))
        h2 = nn.functional.relu(self.l2(h1))
        do = self.do(h2 + h1)  # residual connection
        logits = self.l3(do)
        return logits

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered container of modules; good for easy and quick networks. No need to specify forward method!

In [ ]:
# defining model via sequential
# shorthand, no need for forward method
model_seq = nn.Sequential(
    nn.Linear(28 * 28, 64),
    nn.ReLU(),
    nn.Linear(64, 64),
    nn.ReLU(),
    nn.Dropout(0.1),  # often helps with overfitting
    nn.Linear(64, 10),
)

In [ ]:
# move model to GPU/device memory
model = model_seq.to(device)

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing nn.Module automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s parameters() or named_parameters() methods.

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

In [ ]:
# define loss function
loss = nn.CrossEntropyLoss()  # softmax + neg. log

# 4 Backpropagation via Autograd

![Computational graph](https://pytorch.org/tutorials/_images/comp-graph.png)

In a forward pass, autograd does two things simultaneously:

- run the requested operation to compute a resulting tensor

- maintain the operation’s gradient function in the DAG.

The backward pass kicks off when .backward() is called on the DAG root. autograd then:

- computes the gradients from each .grad_fn,

- accumulates them in the respective tensor’s .grad attribute

- using the chain rule, propagates all the way to the leaf tensors.

# 5 Optimization of model parameters (training)

We define the following hyperparameters for training:

- Number of Epochs - the number times to iterate over the dataset
- Batch Size - the number of data samples propagated through the network before the parameters are updated (defined in train_Loader)
- Learning Rate - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during training.

In [ ]:
lr = 1e-2
epochs = 5

In [ ]:
# defining optimizer
params = model.parameters()
optimiser = optim.SGD(params, lr=1e-2)

Inside the training loop, optimization happens in three steps:

- Call **optimizer.zero_grad()** to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
- Backpropagate the prediction loss with a call to **loss.backward()**. PyTorch deposits the gradients of the loss w.r.t. each parameter.
- Once we have our gradients, we call **optimizer.step()** to adjust the parameters by the gradients collected in the backward pass.

In [ ]:
# define training and validation loop
# training loop
for epoch in range(epochs):
    losses = list()
    accuracies = list()
    model.train()  # enables dropout/batchnorm
    for batch in train_loader:
        x, y = batch
        batch_size = x.size(0)
        # x: b x 1 x 28 x 28
        x = x.view(batch_size, -1).to(device)

        # 5 steps to train network
        # 1 forward
        l = model(x)  # l: logits

        # 2 compute objective function
        J = loss(l, y.to(device))

        # 3 cleaning the gradients (could also call this on optimiser)
        model.zero_grad()
        # optimizer.zero_grad() is equivalent
        # manually: params.grad._zero()

        # 4 accumulate the partial derivatives of J wrt params
        J.backward()
        # manually: params.grad.add_(dJ/dparams)

        # 5 step in the opposite direction of the gradient
        optimiser.step()
        # could have done manual gradient update:
        # with torch.no_grad():
        #   params = params - lr * params.grad
        losses.append(J.item())
        accuracies.append(y.eq(l.detach().argmax(dim=1).cpu()).float().mean())

    print(f"epoch {epoch + 1}", end=", ")
    print(f"training loss: {torch.tensor(losses).mean():.2f}", end=", ")
    print(
        f"training accuracy: {torch.tensor(accuracies).mean():.2f}"
    )  # print two decimals

    # validation loop
    losses = list()
    accuracies = list()
    model.eval()  # disables dropout/batchnorm
    for batch in val_loader:
        x, y = batch
        batch_size = x.size(0)
        # x: b x 1 x 28 x 28
        x = x.view(batch_size, -1).to(device)

        # 5 steps to train network
        # 1 forward
        with torch.no_grad():  # more efficient, just tensor no graph connected
            l = model(x)  # l: logits

        # 2 compute objective function
        J = loss(l, y.to(device))
        losses.append(J.item())
        accuracies.append(y.eq(l.detach().argmax(dim=1).cpu()).float().mean())

    print(f"epoch {epoch + 1}", end=", ")
    print(f"validation loss: {torch.tensor(losses).mean():.2f}", end=", ")
    print(
        f"validation accuracy: {torch.tensor(accuracies).mean():.2f}"
    )  # print two decimals

# 6 Store models


In [ ]:
# just save model weights without structure
torch.save(model.state_dict(), "model_weights.pth")
model.load_state_dict(torch.load("model_weights.pth"))
model.eval()

In [ ]:
# save whole model
torch.save(model, "model.pth")
new_model = torch.load("model.pth")

# 7 Material sources/more resources:
- [PyTorch Tutorial Page](https://pytorch.org/tutorials/beginner/basics/)
- [YouTube Series for PyTorch Lightning](https://www.youtube.com/watch?v=OMDn66kM9Qc)
- [PyTorch common mistakes video from Alladin Persson](https://www.youtube.com/watch?v=O2wJ3tkc-TU&list=PLhhyoLH6IjfxeoooqP9rhU3HJIAVAJ3Vz&index=14)

Extra: interactive debugging during training loop

In [ ]:
# import pdb; pdb.set_trace() (put these commands after logit computation in training loop)
# interactive debugger, leave via q
# you can call the following commands interactively during the training loop
# p l.size()
# p l[0]
# p l[0].detach().argmax()
# p l[0].detach().softmax(dim=0)
# p [f"{prob:.2f}" for prob in l[0].detach().softmax(dim=0)
# p y[:4]
# p l.detach().argmax(dim=1)[:4]
# p y[:4].eq(l.detach().argmax(dim=1)[:4])
# p y.eq(l.detach().argmax(dim=1)).float().mean()